## Imports

In [2]:
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import os
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torchvision.models import resnet50, ResNet50_Weights

## Neural Network

In [3]:
class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(6144, 1024)
        self.fc2 = nn.Linear(1024, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)

    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        return x

## Create embedding

In [4]:
# generate embedding for each image in the dataset
if(os.path.exists('dataset/embeddings.npy') == False):
    weights = ResNet50_Weights.DEFAULT
    train_transforms = transforms.Compose([transforms.ToTensor()
                                       , transforms.Resize(256) 
                                       , transforms.CenterCrop(224)
                                       , transforms.Normalize([0.6110, 0.5012, 0.3752], [0.2575, 0.2659, 0.2801])
                                       ])

    train_dataset = datasets.ImageFolder(root="dataset/", transform=train_transforms)
    
    train_loader = DataLoader(dataset=train_dataset,
                                batch_size=50, #previous 64
                                shuffle=False, num_workers=8)

    model = resnet50(weights=weights)
    model.eval()
    
    for param in model.parameters():
        param.requires_grad = False

    model.fc = nn.Sequential()

    embeddings = []

    for features, labels in train_loader:
        embeddings.append(model(features).T.numpy())  

    np.save('dataset/embeddings.npy', embeddings)

## Load data

In [5]:
def get_data(file, train=True):

    triplets = []
    with open(file) as f:
        for line in f:
            triplets.append(line)
        
    train_dataset = datasets.ImageFolder(root="dataset/",
                                         transform=None)
    filenames = [s[0].split('/')[-1].replace('.jpg', '').replace('food\\', '') for s in train_dataset.samples]

    embeddings = np.swapaxes(np.load('dataset/embeddings.npy'), 1, 2)

    file_to_embedding = {}
    for i in range(200):
        for j in range(50):
            file_to_embedding[filenames[j+i*50]] = embeddings[i][j]
    X = []
    y = []

    for t in triplets:
        emb = [file_to_embedding[a] for a in t.split()]
        X.append(np.hstack([emb[0], emb[1], emb[2]]))
        y.append([[1]])
        if train:
            X.append(np.hstack([emb[0], emb[2], emb[1]]))
            y.append([[0]])
    X = np.vstack(X)
    y = np.hstack(y).T
    return X, y

def create_loader_from_np(X, y = None, train = True, batch_size=64, shuffle=True, num_workers = 8):

    if train:
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float), 
                                torch.from_numpy(y).type(torch.float))
    else:
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float))
    loader = DataLoader(dataset=dataset,
                        batch_size=batch_size,
                        shuffle=shuffle,
                        pin_memory=True, num_workers=num_workers)
    return loader


TRAIN_TRIPLETS = 'train_triplets.txt'
TEST_TRIPLETS = 'test_triplets.txt'

# load the training and testing data
X, y = get_data(TRAIN_TRIPLETS)
X_test, _ = get_data(TEST_TRIPLETS, train=False)

train_loader = create_loader_from_np(X, y, train = True, batch_size=64)
test_loader = create_loader_from_np(X_test, train = False, batch_size=2048, shuffle=False)

## Training

In [ ]:
# Training
model = Net()
n_epochs = 5

optimizer = optim.SGD(model.parameters(), lr=0.01)
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.7, total_iters=n_epochs)
criterion = nn.MSELoss()

for epoch in range(n_epochs):
    print(f'\nepoch={epoch}')
    running_loss = 0.0      
    for i, [X, y] in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 500 == 499:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 500:.3f}')
            running_loss = 0.0

    scheduler.step()


## Testing

In [7]:
# Testing

model.eval()
predictions = []

with torch.no_grad():
    for [X] in test_loader:
        predicted = model(X)
        predicted[predicted >= 0.5] = 1
        predicted[predicted < 0.5] = 0
        predictions.append(predicted)
    predictions = np.vstack(predictions)
    
np.savetxt("results.txt", predictions, fmt='%i')
print("Results saved to results.txt")

Results saved to results.txt
